# Análise Exploratória dos dados da IMDb

Mini-Projeto utilizando Linguagem Python, SQL e Banco de Dados SQLite para fazer uma análise exploratória dos dados do IMDb.Este projeto tem como objetivo realizar uma Análise Exploratória de Dados (EDA) em um conjunto de dados do IMDb. A análise explora diversas variáveis relacionadas a filmes, incluindo notas, gêneros e ano de lançamento, utilizando bibliotecas do ecossistema Python para manipulação e visualização de dados.

## Objetivos
Carregar e explorar o conjunto de dados do IMDb.
Identificar padrões e relações entre variáveis.
Criar visualizações gráficas para facilitar a compreensão dos dados.
Gerar insights sobre tendências em avaliações de filmes.

## Instalando e Carregando Pacotes

In [ ]:
# Instala o pacote
!pip install -q imdb-sqlite

Download dos datasets.

In [ ]:
# Instala o pacote
# https://pypi.org/project/pycountry/
!pip install -q pycountry

In [ ]:
# Imports
import re
import time
import sqlite3
import pycountry
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings("ignore")
sns.set_theme(style = "whitegrid")

## Carregando os Dados

Download dos dados

In [ ]:
%%time
!imdb-sqlite

In [ ]:
# Conectando no banco de dados
conn = sqlite3.connect("imdb.db")

In [ ]:
# Extraindo a lista de tabelas
tabelas = pd.read_sql_query("SELECT NAME AS 'Table_Name' FROM sqlite_master WHERE type = 'table'", conn)

In [ ]:
# Tipo do objeto
type(tabelas)

In [ ]:
# Visualizando o resultado
tabelas.head()

In [ ]:
# Convertendo o dataframe em uma lista
tabelas = tabelas["Table_Name"].values.tolist()

In [ ]:
# Extraindo o esquema da tabela
for tabela in tabelas:
    consulta = "PRAGMA TABLE_INFO({})".format(tabela)
    resultado = pd.read_sql_query(consulta, conn)
    print("Esquema da tabela:", tabela)
    display(resultado)
    print("-"*100)
    print("\n")

# Análise Exploratória dos Dados.

## 1- Quais São as Categorias de Filmes Mais Comuns no IMDB?

Quais são os principais tipos (categorias) dos títulos (filmes)?

In [ ]:
# Criando a consulta SQL
consulta1 = '''SELECT type, COUNT(*) AS COUNT FROM titles GROUP BY type''' 

In [ ]:
# Extraindo o resultado
resultado1 = pd.read_sql_query(consulta1, conn)

In [ ]:
# Visualização do resultado
display(resultado1)

In [ ]:
# Calculando o percentual para cada tipo
resultado1['percentual'] = (resultado1['COUNT'] / resultado1['COUNT'].sum()) * 100

In [ ]:
# Visualização do resultado
display(resultado1)

In [ ]:
# Vamos criar um gráfico com apenas 4 categorias:
# As 3 categorias com mais títulos e 1 categoria com todo o restante

# Criando um dicionário vazio
others = {}

# Filtrando o percentual em 5% e soma o total
others['COUNT'] = resultado1[resultado1['percentual'] < 5]['COUNT'].sum()

# Gravando o percentual
others['percentual'] = resultado1[resultado1['percentual'] < 5]['percentual'].sum()

# Ajustando o nome
others['type'] = 'others'

In [ ]:
# Visualização
others

In [ ]:
# Filtrando o dataframe de resultado
resultado1 = resultado1[resultado1['percentual'] > 5]

In [ ]:
# Append com o dataframe de outras categorias
resultado1 = resultado1.append(others, ignore_index = True)

In [ ]:
# Ordenando o resultado
resultado1 = resultado1.sort_values(by = 'COUNT', ascending = False)

In [ ]:
# Visualização
resultado1.head()

In [ ]:
# Ajustando os labels
labels = [str(resultado1['type'][i])+' '+'['+str(round(resultado1['percentual'][i],2)) +'%'+']' for i in resultado1.index]

In [ ]:
# Plot

# Mapa de cores
# https://matplotlib.org/stable/tutorials/colors/colormaps.html
cs = cm.Set3(np.arange(100))

# Criando a figura
f = plt.figure()

# Pie Plot
plt.pie(resultado1['COUNT'], labeldistance = 1, radius = 3, colors = cs, wedgeprops = dict(width = 0.8))
plt.legend(labels = labels, loc = 'center', prop = {'size':12})
plt.title("Distribuição de Títulos", loc = 'Center', fontdict = {'fontsize':20,'fontweight':20})
plt.show()

Ao analisar os dados da IMDb, observa-se que 73,14% do conteúdo é classificado como TV Episode, seguido de shorts (curtas-metragens) com 10,16%, e apenas 7,21% corresponde à categoria movie (filmes). Esse panorama revela um dado interessante: a maioria do conteúdo cadastrado na plataforma não são filmes tradicionais, como muitas vezes se supõe ao pensar na IMDb. Entretanto, o grande número de TV Episode pode ser explicado pela forte presença e relevância das produções seriadas, que incluem temporadas longas, spin-offs e minisséries.

## 2- Qual o Número de Títulos Por Gênero?

Vamos calcular o número de filmes por gênero e entregar o resultado em valor percentual.

In [ ]:
# Criado a consulta SQL
consulta2 = '''SELECT genres, COUNT(*) FROM titles WHERE type = 'movie' GROUP BY genres''' 

In [ ]:
# Resultado
resultado2 = pd.read_sql_query(consulta2, conn)

In [ ]:
# Visualização do resultado
display(resultado2)

In [ ]:
# Convertendo as strings para minúsculo
resultado2['genres'] = resultado2['genres'].str.lower().values

In [ ]:
# Removendo valores NA (ausentes)
temp = resultado2['genres'].dropna()

Usamos o Countvectorizer para converter a coluna de gêneros em um vetor one-hot encoded para contar o número de filmes em cada gênero.

In [ ]:
# Criando um vetor usando expressão regular para filtrar as strings

# https://docs.python.org/3.8/library/re.html
padrao = '(?u)\\b[\\w-]+\\b'

# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
vetor = CountVectorizer(token_pattern = padrao, analyzer = 'word').fit(temp)

In [ ]:
type(vetor)

In [ ]:
# Aplicando a vetorização ao dataset sem valores NA
bag_generos = vetor.transform(temp)

In [ ]:
type(bag_generos)

In [ ]:
# Retornando gêneros únicos
generos_unicos =  vetor.get_feature_names()

In [ ]:
# Criando o dataframe de gêneros
generos = pd.DataFrame(bag_generos.todense(), columns = generos_unicos, index = temp.index)

In [ ]:
# Visualização
generos.info()

In [ ]:
# Drop da coluna n
generos = generos.drop(columns = 'n', axis = 0)

In [ ]:
# Calculando o percentual
generos_percentual = 100 * pd.Series(generos.sum()).sort_values(ascending = False) / generos.shape[0]

In [ ]:
# Visualização
generos_percentual.head(10)

In [ ]:
# Plot
plt.figure(figsize = (16,8))
sns.barplot(x = generos_percentual.values, y = generos_percentual.index, orient = "h", palette = "terrain")
plt.ylabel('Gênero')             
plt.xlabel("\nPercentual de Filmes (%)")
plt.title('\nNúmero (Percentual) de Títulos Por Gênero\n')
plt.show()

Ao analisar a distribuição percentual de títulos por gênero, fica evidente a predominância do gênero Drama, que representa a maior fatia dos títulos cadastrados na IMDb, seguido por Comédia e Documentário. Em relação aos gêneros com menor participação nas produções são: Game-Show, Talk-Show, Film-Noir e Reality-TV.

## 3- Qual a Mediana de Avaliação dos Filmes Por Gênero?

A seguir temos os resultados para a análise das avalições dos filmes. Apesar do produção de filmes ser predominantemente do gênero drama e seguido de comédia, o que possui a melhor avaliação mediana são os filmes do gênero documentário, ou seja, 50% das avalições desse gênero são maiores ou iguais a 7,3 e 50% são menores ou iguais a esse valor.

In [ ]:
# Consulta SQL
consulta3 = '''
            SELECT rating, genres FROM 
            ratings JOIN titles ON ratings.title_id = titles.title_id 
            WHERE premiered <= 2022 AND type = 'movie'
            ''' 

In [ ]:
# Resultado
resultado3 = pd.read_sql_query(consulta3, conn)

In [ ]:
# Visualização
display(resultado3)

In [ ]:
# Criando uma função para retornar os genêros
def retorna_generos(df):
    df['genres'] = df['genres'].str.lower().values
    temp = df['genres'].dropna()
    vetor = CountVectorizer(token_pattern = '(?u)\\b[\\w-]+\\b', analyzer = 'word').fit(temp)
    generos_unicos =  vetor.get_feature_names()
    generos_unicos = [genre for genre in generos_unicos if len(genre) > 1]
    return generos_unicos

In [ ]:
# Aplicando a função
generos_unicos = retorna_generos(resultado3)

In [ ]:
# Visualização
generos_unicos

In [ ]:
# Criando listas vazias
genero_counts = []
genero_ratings = []

In [ ]:
# Loop
for item in generos_unicos:
    
    # Retorna a contagem de filmes por gênero
    consulta = 'SELECT COUNT(rating) FROM ratings JOIN titles ON ratings.title_id=titles.title_id WHERE genres LIKE '+ '\''+'%'+item+'%'+'\' AND type=\'movie\''
    resultado = pd.read_sql_query(consulta, conn)
    genero_counts.append(resultado.values[0][0])
  
     # Retorna a avaliação de filmes por gênero
    consulta = 'SELECT rating FROM ratings JOIN titles ON ratings.title_id=titles.title_id WHERE genres LIKE '+ '\''+'%'+item+'%'+'\' AND type=\'movie\''
    resultado = pd.read_sql_query(consulta, conn)
    genero_ratings.append(np.median(resultado['rating']))

In [ ]:
# Preparação do dataframe final
df_genero_ratings = pd.DataFrame()
df_genero_ratings['genres'] = generos_unicos
df_genero_ratings['count'] = genero_counts
df_genero_ratings['rating'] = genero_ratings

In [ ]:
# Visualização
df_genero_ratings.head(20)

In [ ]:
# Drop do índice 18 (news)
# Não queremos essa informação como gênero
df_genero_ratings = df_genero_ratings.drop(index = 18)

In [ ]:
# Ordenando o resultado
df_genero_ratings = df_genero_ratings.sort_values(by = 'rating', ascending = False)

In [ ]:
# Plot

# Figura
plt.figure(figsize = (16,10))

# Barplot
sns.barplot(y = df_genero_ratings.genres, x = df_genero_ratings.rating, orient = "h")

# Textos do gráfico
for i in range(len(df_genero_ratings.index)):
    
    plt.text(4.0, 
             i + 0.25, 
             str(df_genero_ratings['count'][df_genero_ratings.index[i]]) + " filmes")
    
    plt.text(df_genero_ratings.rating[df_genero_ratings.index[i]],
             i + 0.25,
             round(df_genero_ratings["rating"][df_genero_ratings.index[i]],2))

plt.ylabel('Gênero')             
plt.xlabel('Mediana da Avaliação')
plt.title('\nMediana de Avaliação Por Gênero\n')
plt.show()

## 4- Qual a Mediana de Avaliação dos Filmes Em Relação ao Ano de Estréia?

Mediana de avaliação dos filmes.

In [ ]:
# Consulta SQL
consulta4 = '''
            SELECT rating AS Rating, premiered FROM 
            ratings JOIN titles ON ratings.title_id = titles.title_id 
            WHERE premiered <= 2022 AND type = 'movie'
            ORDER BY premiered
            ''' 

In [ ]:
# Resultado
resultado4 = pd.read_sql_query(consulta4, conn)

In [ ]:
display(resultado4)

In [ ]:
# Calculamos a mediana ao longo do tempo (anos)
ratings = []
for year in set(resultado4['premiered']):
    ratings.append(np.median(resultado4[resultado4['premiered'] == year]['Rating']))

In [ ]:
type(ratings)

In [ ]:
ratings[1:10]

In [ ]:
# Lista de anos
anos = list(set(resultado4['premiered']))

In [ ]:
anos[1:10]

In [ ]:
# Plot
plt.figure(figsize = (16,8))
plt.plot(anos, ratings)
plt.xlabel('\nAno')
plt.ylabel('Mediana de Avaliação')
plt.title('\nMediana de Avaliação dos Filmes Em Relação ao Ano de Estréia\n')
plt.show()

Ao analisar a mediana das avalições por ano de estréia, pode-se observar que nos primeiros anos há uma grande oscilação entre as medianas das avaliações. Somente, a partir de 1920, nota-se uma estabilização gradual, permanecendo estável acima de 6,0 e 6,5. Entretanto, a partir dos anos 2000, a tendência é de aumento da mediana, permanecendo mais próximo dos 6,5 do que dos 6,0. Portanto, nesse gráfico temos uma tendência positiva nas avalições dos filmes, podendo ser justificado pelo maior investimento em tecnologia, produção e roteiro.

## 5- Qual o Número de Filmes Avaliados Por Gênero Em Relação ao Ano de Estréia?

O gráfico a seguir mostra o número de filmes avaliados por gênero em relação ao ano de estréia. Para essa análise destacamos os gêneros mais avaliados, que são: drama, documentário, comédia, ação e romance. Para o período analisado, o gênero drama possui predominancia na avaliação. Destacando o período após os anos de 2000 que houve aumento expressivo na quantidade de filmes avaliados desse gênero e de outros, e em 2017, drama ultrapassa mais de 6 mil filmes avaliados. Até aproximadamente os anos 2000, o segundo gênero mais avalido era documentário, sendo ultrapassando pelo gênero comédia e atingindo valores bem próximos da quantidade de filmes avalidos do gênero drama. Já os gêneros ação e romance, ficam mais proximos de mil filmes avaliados. Para o período após 2020, observa-se uma queda no número de avaliações, que pode ser explicada por dois fatores principais: o impacto da pandemia da COVID-19 e do atraso nas avalições dos títulos mais recentes.

In [ ]:
# Consulta SQL
consulta5 = '''SELECT genres FROM titles ''' 

In [ ]:
# Resultado
resultado5 = pd.read_sql_query(consulta5, conn)

In [ ]:
display(resultado5)

In [ ]:
# Retorna gêneros únicos
generos_unicos = retorna_generos(resultado5)

In [ ]:
# Visualização do resultado
generos_unicos

In [ ]:
# Fazendo a contagem
genero_count = []
for item in generos_unicos:
    consulta = 'SELECT COUNT(*) COUNT FROM  titles  WHERE genres LIKE '+ '\''+'%'+item+'%'+'\' AND type=\'movie\' AND premiered <= 2022'
    resultado = pd.read_sql_query(consulta, conn)
    genero_count.append(resultado['COUNT'].values[0])

In [ ]:
# Preparando o dataframe
df_genero_count = pd.DataFrame()
df_genero_count['genre'] = generos_unicos
df_genero_count['Count'] = genero_count

In [ ]:
# Calculando os top 5
df_genero_count = df_genero_count[df_genero_count['genre'] != 'n']
df_genero_count = df_genero_count.sort_values(by = 'Count', ascending = False)
top_generos = df_genero_count.head()['genre'].values

In [ ]:
# Plot

# Figura
plt.figure(figsize = (16,8))

# Loop e Plot
for item in top_generos:
    consulta = 'SELECT COUNT(*) Number_of_movies, premiered Year FROM  titles  WHERE genres LIKE '+ '\''+'%'+item+'%'+'\' AND type=\'movie\' AND Year <=2022 GROUP BY Year'
    resultado = pd.read_sql_query(consulta, conn)
    plt.plot(resultado['Year'], resultado['Number_of_movies'])

plt.xlabel('\nAno')
plt.ylabel('Número de Filmes Avaliados')
plt.title('\nNúmero de Filmes Avaliados Por Gênero Em Relação ao Ano de Estréia\n')
plt.legend(labels = top_generos)
plt.show()

## 6- Qual o Filme Com Maior Tempo de Duração? Calcule os Percentis.

Percentis e filme mais longo da história.

In [ ]:
# Consulta SQL
consulta6 = '''
            SELECT runtime_minutes Runtime 
            FROM titles 
            WHERE type = 'movie' AND Runtime != 'NaN'
            ''' 

In [ ]:
# Resultado
resultado6 = pd.read_sql_query(consulta6, conn)

In [ ]:
display(resultado6)

In [ ]:
# Loop para cálculo dos percentis
for i in range(101): 
    val = i
    perc = round(np.percentile(resultado6['Runtime'].values, val), 2)
    print('{} percentil da duração (runtime) é: {}'.format(val, perc))

In [ ]:
# Refazendo a consulta e retornando o filme com maior duração
consulta6 = '''
            SELECT runtime_minutes Runtime, primary_title
            FROM titles 
            WHERE type = 'movie' AND Runtime != 'NaN'
            ORDER BY Runtime DESC
            LIMIT 1
            ''' 

In [ ]:
resultado6 = pd.read_sql_query(consulta6, conn)

In [ ]:
resultado6

Em relação a duração dos filmes, o filme mais longo, até o momento, é 'Logistics', com 857 horas (51420 minutos). Dirigido por Daniel Andersson e Erika Magnusson, o filme acompanha todo o ciclo de vida de um pedômetro (dispositivo de rastreamento de passos) em ordem cronológica reversa. Esse filme é do gênero documentário, entretanto esse gênero não é considerado o maior em relação a duração. 

## 7- Qual a Relação Entre Duração e Gênero?
O gênero com maior duração é romance com 95 minutos, seguido por ação, guerra e drama (93 min). Sendo, a maioria dos longas-metragens com 90 minutos ou mais. Já o gênero documentário tem duração de 74 minutos, mostrando o diferencial do filme Logistics, que apresenta mais de 694 vezes a duração mediana do gênero.

In [ ]:
# Consulta SQL
consulta7 = '''
            SELECT AVG(runtime_minutes) Runtime, genres 
            FROM titles 
            WHERE type = 'movie'
            AND runtime_minutes != 'NaN'
            GROUP BY genres
            ''' 

In [ ]:
# Resultado
resultado7 = pd.read_sql_query(consulta7, conn)

In [ ]:
# Retornando gêneros únicos
generos_unicos = retorna_generos(resultado7)

In [ ]:
# Visualização
generos_unicos

In [ ]:
# Calculando duração por gênero
genero_runtime = []
for item in generos_unicos:
    consulta = 'SELECT runtime_minutes Runtime FROM  titles  WHERE genres LIKE '+ '\''+'%'+item+'%'+'\' AND type=\'movie\' AND Runtime!=\'NaN\''
    resultado = pd.read_sql_query(consulta, conn)
    genero_runtime.append(np.median(resultado['Runtime']))

In [ ]:
# Preparando o dataframe
df_genero_runtime = pd.DataFrame()
df_genero_runtime['genre'] = generos_unicos
df_genero_runtime['runtime'] = genero_runtime

In [ ]:
# Removendo índice 18 (news)
df_genero_runtime = df_genero_runtime.drop(index = 18)

In [ ]:
# Ordenando os dados
df_genero_runtime = df_genero_runtime.sort_values(by = 'runtime', ascending = False)

In [ ]:
# Plot

# Tamanho da figura
plt.figure(figsize = (16,8))

# Barplot
sns.barplot(y = df_genero_runtime.genre, x = df_genero_runtime.runtime, orient = "h")

# Loop
for i in range(len(df_genero_runtime.index)):
    plt.text(df_genero_runtime.runtime[df_genero_runtime.index[i]],
             i + 0.25,
             round(df_genero_runtime["runtime"][df_genero_runtime.index[i]], 2))

plt.ylabel('Gênero')             
plt.xlabel('\nMediana de Tempo de Duração (Minutos)')
plt.title('\nRelação Entre Duração e Gênero\n')
plt.show()

## 8- Qual o Número de Filmes Produzidos Por País?

Número de filmes produzidos por país.

In [ ]:
# Consulta SQL
consulta8 = '''
            SELECT region, COUNT(*) Number_of_movies FROM 
            akas JOIN titles ON 
            akas.title_id = titles.title_id
            WHERE region != 'None'
            AND type = \'movie\'
            GROUP BY region
            ''' 

In [ ]:
# Resultado
resultado8 = pd.read_sql_query(consulta8, conn)

In [ ]:
display(resultado8)

In [ ]:
# Shape
resultado8.shape

In [ ]:
# Número de linhas
resultado8.shape[0]

In [ ]:
# Listas auxiliares
nomes_paises = []
contagem = []

In [ ]:
# Loop para obter o país de acordo com a região
for i in range(resultado8.shape[0]):
    try:
        coun = resultado8['region'].values[i]
        nomes_paises.append(pycountry.countries.get(alpha_2 = coun).name)
        contagem.append(resultado8['Number_of_movies'].values[i])
    except: 
        continue

In [ ]:
# Preparando o dataframe
df_filmes_paises = pd.DataFrame()
df_filmes_paises['country'] = nomes_paises
df_filmes_paises['Movie_Count'] = contagem

In [ ]:
# Ordenando o resultado
df_filmes_paises = df_filmes_paises.sort_values(by = 'Movie_Count', ascending = False)

In [ ]:
# Visualização
df_filmes_paises.head(10)

In [ ]:
# Plot

# Figura
plt.figure(figsize = (20,8))

# Barplot
sns.barplot(y = df_filmes_paises[:20].country, x = df_filmes_paises[:20].Movie_Count, orient = "h")

# Loop
for i in range(0,20):
    plt.text(df_filmes_paises.Movie_Count[df_filmes_paises.index[i]]-1,
             i + 0.30,
             round(df_filmes_paises["Movie_Count"][df_filmes_paises.index[i]],2))

plt.ylabel('País')             
plt.xlabel('\nNúmero de Filmes')
plt.title('\nNúmero de Filmes Produzidos Por País\n')
plt.show()

Os três maiores países em produção de filmes são: Estados Unidos com 255.987 filmes, seguido pelo Reino Unido com 113.225 e o Japão com 87.815 filmes. A diferença entre os Estados Unidos e o segundo colocado ultrapassa 142 mil filmes, ou seja, mais de duas vezes a produção do Reino Unido, o que reforça o tamanho, a influência e o investimento contínuo da indústria cinematográfica norte-americana no cenário global. Comparando com o Brasil, que produz 61.825 filmes e ocupa a 9ª posição, a produção cinematográfica nacional representa cerca de 24% da produção norte-americana.

## 9- Quais São os Top 10 Melhores Filmes?

Top 10 filmes com melhor avaliação e mais de 25 mil votos.

In [ ]:
# Consulta SQL
consulta9 = '''
            SELECT primary_title AS Movie_Name, genres, rating
            FROM 
            titles JOIN ratings
            ON  titles.title_id = ratings.title_id
            WHERE titles.type = 'movie' AND ratings.votes >= 25000
            ORDER BY rating DESC
            LIMIT 10          
            ''' 

In [ ]:
# Resultado
top10_melhores_filmes = pd.read_sql_query(consulta9, conn)

In [ ]:
display(top10_melhores_filmes)

Ao analisar as avaliações dos filmes, o filme com a maior nota é The Shawshank Redemption, seguido de The Chaos Class e The Godfather, ambos do gênero Drama. O gênero Drama também aparece em 8 dos 10 melhores filmes, podendo indicar a preferência por uma grande carga emocional.

## 10- Quais São os Top 10 Piores Filmes?

Top 10 filmes com pior avaliação e mais de 25 mil votos.

In [ ]:
# Consulta SQL
consulta10 = '''
            SELECT primary_title AS Movie_Name, genres, rating
            FROM 
            titles JOIN ratings
            ON  titles.title_id = ratings.title_id
            WHERE titles.type = 'movie' AND ratings.votes >= 25000
            ORDER BY rating ASC
            LIMIT 10
            ''' 

In [ ]:
# Resultado
top10_piores_filmes = pd.read_sql_query(consulta10, conn)

In [ ]:
display(top10_piores_filmes)

Já os filmes com menor aceitação foram: o filme Cumali Ceber, do gênero comédia, seguido de Smolensk e Sadak 2, sendo esses dois últimos do gênero drama. Esses títulos ocupam as posições mais baixas da tabela, com notas entre 1.0 e 1.1, indicando forte rejeição do público e da crítica.

Tanto os melhores filmes e piores filmes, são na sua maioria, do gênero drama ou comédia. Podendo significar que quando mal executados, sofrem maior rejeição, reforçando a importância de um bom desenvolvimento de roteiro, direção e atuação para agradar ao público.

# Conclusão
Conclui-se que a partir dos anos 2000, houve um aumento significativo na produção cinematográfica e que os gêneros mais produzidos são drama e comédia. Entretanto, no quesito aceitação do público, o gênero documentário é o mais bem avaliado. Sendo que avaliação não está diretamente relacionada com a duração do filme. Outro ponto a destacar, é a representatividade da produção dos Estados Unidos, sendo esta mais de duas vezes a quantidade produzido pelo segundo país do ranking.